In [ ]:
# initial setup
try:
    # settings colab:
    import google.colab

    ! mkdir -p ../Data
    # los que usan colab deben modificar el token de esta url:
    ! wget -O ../Data/bikes.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M3/CLASE_16_Regresion_Lineal_Multiple/Data/bikes.csv?token=AA4GFHIG665I3BPVQCFY3US63APZM
    
except ModuleNotFoundError:    
    # settings local:
    %run "common/0_notebooks_base_setup.py"

In [ ]:
import pandas as pd
import numpy as np
import sklearn.preprocessing as preprocessing
import sklearn.model_selection as model_selection
import sklearn.metrics as metrics
import sklearn.linear_model as linear_model
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import re
import unicodedata
%matplotlib inline

# Definimos parámetros globales para matplotlib.
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 16

data_original = pd.read_csv('Clean08_OultiersxZona_Ale_02.csv')

In [ ]:
data_original.head(2)

In [ ]:
data_original.columns.values

In [ ]:
for c in data_original.columns[3:]:
    if data_original[c].isnull().sum() > 0:
        print(c, " ", data_original[c].isnull().sum())

In [ ]:
data = data_original.drop(columns=
['Unnamed: 0', 
 'Unnamed: 0.1', 
 'Unnamed: 0.1.1',
 'index',
 'operation', 
 'property_type', 
 'geonames_id', 
 'lat-lon',
 'price',
 'currency',
 'country_name',
 'price_aprox_local_currency',
 'price_aprox_usd',
 'price_per_m2',
 'floor', 
 'title', 
 'image_thumbnail', 
 'Operacion_Description',
 'expenses',
 'properati_url',
 'Operacion_Title', 
 'Operacion_URL',
 'url_clean',
 'url_terraza', 
 'url_quincho', 
 'desc_terraza', 
 'desc_quincho',
 'desc_banios',
 'tipo_propiedad_Fondo de Comercio', 
 'Outliers',
 'País',
 'Provincia', 
 'Área', 
 'Localidad', 
 'Zona', 
 'Lugar',
 'place_with_parent_names', 
 'lat', 'lon', 'place_name', 'state_name'])

In [ ]:
largo_maximo_zona = data_original.place_with_parent_names.apply(lambda x: len(x[1:len(x) - 1].split("|"))).max()

In [ ]:
def agregar_ubicacion(zona_str, largo_maximo_zona):
    zona_array = zona_str[1:len(zona_str) - 1].split("|")
    largo = len(zona_array)
    if largo < largo_maximo_zona:
        zona_array.extend(np.full(largo_maximo_zona - largo, zona_array[largo - 1]))
    return zona_array

In [ ]:
zonas_serie = data_original.place_with_parent_names.apply(lambda x: agregar_ubicacion(x, largo_maximo_zona))

In [ ]:
for i in range(largo_maximo_zona):
    data["zona" + str((i + 1))] = zonas_serie.apply(lambda x: x[i])

In [ ]:
data.head(2)

In [ ]:
#data.lat = np.round(data.lat, 2)
#data.lon = np.round(data.lon, 2)

In [ ]:
print(data.rooms.isnull().sum())
regex = re.compile("\d\sambientes", flags = re.IGNORECASE)
regexRooms = data[(data.rooms.isnull())].description.apply(lambda x: regex.search(str(x)))
resultado = regexRooms.apply(lambda x: np.NaN if x is None else x.group(0).lower().replace(" ambientes", "").strip()).astype(np.float64)
data.loc[resultado.index, "rooms"]  = resultado
print(data.rooms.isnull().sum())

In [ ]:
print(data.rooms.isnull().sum())
regex = re.compile("\d\samb", flags = re.IGNORECASE)
regexRooms = data[(data.rooms.isnull())].description.apply(lambda x: regex.search(str(x)))
resultado = regexRooms.apply(lambda x: np.NaN if x is None else x.group(0).lower().replace(" amb", "").strip()).astype(np.float64)
data.loc[resultado.index, "rooms"]  = resultado
print(data.rooms.isnull().sum())

In [ ]:
print(data.rooms.isnull().sum())
regex = re.compile("1\sambiente", flags = re.IGNORECASE)
regexRooms = data[(data.rooms.isnull())].description.apply(lambda x: regex.search(str(x)))
resultado = regexRooms.apply(lambda x: np.NaN if x is None else x.group(0).lower().replace(" ambiente", "").strip()).astype(np.float64)
data.loc[resultado.index, "rooms"]  = resultado
print(data.rooms.isnull().sum())

In [ ]:
print(data.rooms.isnull().sum())
regex = re.compile("un\sambiente", flags = re.IGNORECASE)
regexRooms = data[(data.rooms.isnull())].description.apply(lambda x: regex.search(str(x)))
resultado = regexRooms.apply(lambda x: np.NaN if x is None else x.group(0).lower().replace("un ambiente", "1").strip()).astype(np.float64)
data.loc[resultado.index, "rooms"]  = resultado
print(data.rooms.isnull().sum())

In [ ]:
data = data.drop(columns=["description"])

In [ ]:
data.columns.values

In [ ]:
# Elimino nulos
data = data.dropna()
data.isnull().sum()

In [ ]:
data.describe()

In [ ]:
feature_cols = data.columns.values

In [ ]:
sns.heatmap(data[feature_cols].corr(), center=0, cmap="YlGnBu", annot=False);

In [ ]:
#prueba = data_original[data_original.property_type == "apartment"].surface_covered_in_m2 / data_original[data_original.property_type == "apartment"].rooms

In [ ]:
data = pd.get_dummies(data, columns=["rooms"], prefix="rooms_", drop_first=True)

In [ ]:
#data = pd.get_dummies(data, columns=["lat"], prefix="lat_", drop_first=True)
#data = pd.get_dummies(data, columns=["lon"], prefix="lon_", drop_first=True)

In [ ]:
for i in range(largo_maximo_zona):
    data = pd.get_dummies(data, columns=["zona" + str(i + 1)], prefix="z" + str(i + 1) + "_", drop_first=True)

In [ ]:
feature_cols = data.columns.values.tolist()
feature_cols.remove('price_usd_per_m2')
predecir = "price_usd_per_m2"

In [ ]:
feature_cols[0:20]

In [ ]:
len(feature_cols)

<hr>

## model_train_test_error_statsmodels

In [ ]:
def model_train_test_error_statsmodels(df, feature_cols, columna_predecir):
    X = df[feature_cols]
    y = df[columna_predecir]
    X_constant = sm.add_constant(X)
    modelo = sm.OLS(y, X_constant).fit()
    y_pred = modelo.predict(X_constant)
    print (modelo.summary())
    print ()
    return modelo, y, X_constant, X

In [ ]:
modeloOLS, yOLS, X_constant, X = model_train_test_error_statsmodels(
    data, 
    feature_cols, predecir)

In [ ]:
feature_cols = modeloOLS.pvalues[modeloOLS.pvalues <= 0.01].index.values[1:]

In [ ]:
len(feature_cols)

<hr>

## model_train_test_error

In [ ]:
# Esta función recibe el modelo ya instanciado con los hiperparámetros, el dataframe, 
# los nombres de las columnas predictoras y el nombre de la columna que quiero predecir
def model_train_test_error(modelo, df, feature_cols, columna_predecir):
    X = df[feature_cols]
    y = df[columna_predecir]
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.30, random_state=42)
    modelo_fit = modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    #print (feature_cols)
    print ("y_test sample: ",y_test.values[0:10])
    print ("y_pred sample: ",y_pred[0:10].astype(int))
    print ('Metrics.MAE:', metrics.mean_absolute_error(y_test, y_pred))
    print ('Metrics.MSE:', metrics.mean_squared_error(y_test, y_pred))
    print ('Metrics.RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print ('Metrics.R2:', metrics.r2_score(y_test, y_pred))
    print ("Modelo_fit.r^2:", modelo_fit.score(X_test, y_test))    
    print ('Modelo_fit.Score test: ', modelo_fit.score(X_test, y_test))
    print ('Modelo_fit.Score train: ', modelo_fit.score(X_train, y_train))
    #print ("Modelo_fit.coef: ", modelo_fit.coef_)

    if hasattr(modelo, 'alpha_'):    
        print("Alpha: ", modelo_fit.alpha_)

    fig, ax = plt.subplots()
    ax.scatter(y_test, y_pred, edgecolors=(0, 0, 0))
    ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
    ax.set_xlabel('Measured')
    ax.set_ylabel('Predicted')
    plt.show()        
        
    print ()
    return

In [ ]:
model_train_test_error(
    linear_model.LinearRegression(), 
    data, 
    feature_cols, predecir)

In [ ]:
model_train_test_error(
    linear_model.Ridge(alpha=0.5, normalize=True), 
    data, 
    feature_cols, predecir)

In [ ]:
model_train_test_error(
    linear_model.Lasso(alpha=0.5, normalize=True), 
    data, 
    feature_cols, predecir)

In [ ]:
model_train_test_error(
    linear_model.ElasticNet(alpha=0.5, normalize=True), 
    data, 
    feature_cols, predecir)

In [ ]:
model_train_test_error(
    linear_model.RidgeCV(alphas=np.linspace(0.01, 10, 20), cv=3, normalize=True,scoring='r2'), 
    data, 
    feature_cols, predecir)

In [ ]:
model_train_test_error(
    linear_model.LassoCV(alphas=np.linspace(0.01, 100, 100), cv=3, normalize=True), 
    data, 
    feature_cols, predecir)

In [ ]:
model_train_test_error(
    linear_model.ElasticNetCV(alphas=np.linspace(0.01, 100, 100), cv=3, normalize=True), 
    data, 
    feature_cols, predecir)

<hr>

## model_train_test_error_CrossVal

In [ ]:
def model_train_test_error_CrossVal(modelo, df, feature_cols, columna_predecir):
    X = df[feature_cols]
    y = df[columna_predecir]
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.30, random_state=42)
    modelo_fit = modelo.fit(X_train, y_train)
    cv = model_selection.KFold(5, shuffle=False)
    scores = model_selection.cross_val_score(modelo, X_train, y_train, cv=cv, scoring='r2')

    if hasattr(modelo, 'alpha_'):
        print(dict(alpha=modelo.alpha_, scores=scores, mean_score=scores.mean(), zero_coefs=(modelo.coef_ == 0).sum()))
    else:
        print("scores: ", scores)
        print("media de scores", scores.mean())
    return 

In [ ]:
model_train_test_error_CrossVal(
    linear_model.LinearRegression(), 
    data, 
    feature_cols, predecir)

In [ ]:
model_train_test_error_CrossVal(
    linear_model.Ridge(normalize=True), 
    data, 
    feature_cols, predecir)

In [ ]:
model_train_test_error_CrossVal(
    linear_model.Lasso(normalize=True), 
    data, 
    feature_cols, predecir)

In [ ]:
model_train_test_error_CrossVal(
    linear_model.ElasticNet(normalize=True), 
    data, 
    feature_cols, predecir)

<hr>

## model_train_test_error_CrossValPredict

In [ ]:
def model_train_test_error_CrossValPredict(modelo, df, feature_cols, columna_predecir):
    X = df[feature_cols]
    y = df[columna_predecir]
    predicted = model_selection.cross_val_predict(modelo, X, y, cv=10)
    print ('Metrics.R2:', metrics.r2_score(y, predicted))
    fig, ax = plt.subplots()
    ax.scatter(y, predicted, edgecolors=(0, 0, 0))
    ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
    ax.set_xlabel('Measured')
    ax.set_ylabel('Predicted')
    plt.show()

In [ ]:
model_train_test_error_CrossValPredict(
    linear_model.LinearRegression(), 
    data, 
    feature_cols, predecir)

In [ ]:
model_train_test_error_CrossValPredict(
    linear_model.Ridge(normalize=True), 
    data, 
    feature_cols, predecir)

In [ ]:
model_train_test_error_CrossValPredict(
    linear_model.Lasso(normalize=True), 
    data, 
    feature_cols, predecir)

In [ ]:
model_train_test_error_CrossValPredict(
    linear_model.ElasticNet(normalize=True), 
    data, 
    feature_cols, predecir)